In [1]:
# importing libraries
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [3]:
# Ensure that you have downloaded the necessary NLTK data
# nltk.download('punkt')
# nltk.download('wordnet')

# lemmatizer = WordNetLemmatizer()

words = []
classes = []
documents = []
ignore_words = ['?', '!']

# Load the intents files
data_file = open('intents.json').read()
intents = json.loads(data_file)

# Process the intents
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # Tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # Add documents in the corpus
        documents.append((w, intent['tag']))
        # Add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

data_file = open('intents1.json').read()
intents = json.loads(data_file)

# Process the second intents file
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # Tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # Add documents in the corpus
        documents.append((w, intent['tag']))
        # Add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Lemmatize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# Sort classes
classes = sorted(list(set(classes)))

# Documents = combination between patterns and intents
print(len(documents), "documents")
# Classes = intents
print(len(classes), "classes", classes)
# Words = all words, vocabulary
print(len(words), "unique lemmatized words", words)

# Save the words and classes to disk
pickle.dump(words, open('texts.pkl', 'wb'))
pickle.dump(classes, open('labels.pkl', 'wb'))

# Create our training data
training = []
# Create an empty array for our output
output_empty = [0] * len(classes)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\xinyu\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\xinyu\AppData\Roaming\nltk_data...


515 documents
131 classes ['What are the types of depression?', 'about', 'afternoon', 'anxious', 'ask', 'at what age does anxiety peak?', 'can lack of sleep make you feel sad?', 'can low blood sugar cause suicidal thoughts?', 'casual', 'creation', 'death', 'default', 'depressed', 'do we control our thoughts?', 'does oversleeping cause depression?', 'done', 'evening', 'fact-1', 'fact-10', 'fact-11', 'fact-12', 'fact-13', 'fact-14', 'fact-15', 'fact-16', 'fact-17', 'fact-18', 'fact-19', 'fact-2', 'fact-20', 'fact-21', 'fact-22', 'fact-23', 'fact-24', 'fact-25', 'fact-26', 'fact-27', 'fact-28', 'fact-29', 'fact-3', 'fact-30', 'fact-31', 'fact-32', 'fact-5', 'fact-6', 'fact-7', 'fact-8', 'fact-9', 'friends', 'goodbye', 'greeting', 'happy', 'hate-me', 'hate-you', 'help', 'how can we reduce anxiety?', 'how does depression affect the world?', 'how long can anxiety last?', 'how many thoughts a day do we have?', 'i am a victim of bullying', 'i am afraid i will fail again', 'i am afraid to file 

In [6]:
# Training set, bag of words for each sentence
for doc in documents:
    # Initialize our bag of words
    bag = []
    # List of tokenized words for the pattern
    pattern_words = doc[0]
    # Lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # Create our bag of words array with 1, if word match found in current pattern
    bag = [1 if w in pattern_words else 0 for w in words]
    
    # Output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

# Shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training, dtype=object)

In [7]:
# Shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training, dtype=object)

# Create train and test lists. X - patterns, Y - intents
train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

print("Training data created")

Training data created


In [8]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))


c:\Users\xinyu\Documents\taskformer\cuda\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

print("Model compiled")

Model compiled


In [12]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('model.keras', hist)
print("model created")

Epoch 1/200
206/206 ━━━━━━━━━━━━━━━━━━━━ 0s 712us/step - accuracy: 0.7790 - loss: 0.9073
Epoch 2/200
206/206 ━━━━━━━━━━━━━━━━━━━━ 0s 691us/step - accuracy: 0.8451 - loss: 0.6806
Epoch 3/200
206/206 ━━━━━━━━━━━━━━━━━━━━ 0s 700us/step - accuracy: 0.8136 - loss: 0.8031
Epoch 4/200
206/206 ━━━━━━━━━━━━━━━━━━━━ 0s 716us/step - accuracy: 0.8325 - loss: 0.8281
Epoch 5/200
206/206 ━━━━━━━━━━━━━━━━━━━━ 0s 754us/step - accuracy: 0.8294 - loss: 0.7532
Epoch 6/200
206/206 ━━━━━━━━━━━━━━━━━━━━ 0s 724us/step - accuracy: 0.8377 - loss: 0.7360
Epoch 7/200
206/206 ━━━━━━━━━━━━━━━━━━━━ 0s 722us/step - accuracy: 0.7945 - loss: 0.9720
Epoch 8/200
206/206 ━━━━━━━━━━━━━━━━━━━━ 0s 719us/step - accuracy: 0.8272 - loss: 0.6967
Epoch 9/200
206/206 ━━━━━━━━━━━━━━━━━━━━ 0s 770us/step - accuracy: 0.8236 - loss: 0.7718
Epoch 10/200
206/206 ━━━━━━━━━━━━━━━━━━━━ 0s 737us/step - accuracy: 0.8044 - loss: 0.9288
Epoch 11/200
206/206 ━━━━━━━━━━━━━━━━━━━━ 0s 678us/step - accuracy: 0.8381 - loss: 0.7505
Epoch 12/200
206/20